In [1]:
import pandas as pd

In [27]:
df = pd.read_csv('/Users/carboni/Downloads/Leopard.csv')

### In case of clusters of other keywords + wkt

In [28]:
transformed_data = []

for cluster, group_df in df.groupby('Cluster'):
    group_df = group_df.sort_values(by='Date')
    group_df['origin'] = group_df['wkt']
    group_df['destination'] = group_df['wkt'].shift(-1).fillna(group_df['wkt'].iloc[-1])
    transformed_data.append(group_df)

# Concatenate all transformed groups
transformed_df = pd.concat(transformed_data)

In [29]:
transformed_df.head()

,Cluster,manifest_url,canvas_number,image_url,City,Country,Title,wkt,Date,Journal Type,Unnamed: 10,origin,destination
2965,Bête Seule (Tigre),https://gallica.bnf.fr/iiif/ark:/12148/bpt6k19...,422.0,https://gallica.bnf.fr/iiif/ark:/12148/bpt6k19...,Paris,France,L'Artiste : journal de la littérature et des b...,POINT(2.3513888888889 48.856944444444),1831-07-10,Art History,NaN,POINT(2.3513888888889 48.856944444444),POINT(2.3513888888889 48.856944444444)
2964,Bête Seule (Tigre),https://gallica.bnf.fr/iiif/ark:/12148/bpt6k19...,412.0,https://gallica.bnf.fr/iiif/ark:/12148/bpt6k19...,Paris,France,L'Artiste : journal de la littérature et des b...,POINT(2.3513888888889 48.856944444444),1832-07-10,Art History,NaN,POINT(2.3513888888889 48.856944444444),POINT(-74.006015 40.712728)
2962,Bête Seule (Tigre),https://iiif.unige.ch/dhportal/ug27806292/mani...,320.0,https://iiif.unige.ch/iiif/2/fedora_ug27812760...,New York,United States of America,The Illustrated Magazine of Art,POINT(-74.006015 40.712728),1854-01-01,Art Journal,NaN,POINT(-74.006015 40.712728),POINT(2.3513888888889 48.856944444444)
2961,Bête Seule (Tigre),https://gallica.bnf.fr/iiif/ark:/12148/bpt6k22...,139.0,https://gallica.bnf.fr/iiif/ark:/12148/bpt6k22...,Paris,France,L'Artiste : journal de la littérature et des b...,POINT(2.3513888888889 48.856944444444),1856-07-10,Art History,NaN,POINT(2.3513888888889 48.856944444444),POINT(-0.1275 51.507222222222)
2959,Bête Seule (Tigre),https://iiif.archive.org/iiif/sim_illustrated-...,23.0,https://iiif.archive.org/image/iiif/3/sim_illu...,London,United Kingdom,The Illustrated London News 1859-05-07: Vol 34...,POINT(-0.1275 51.507222222222),1859-05-07,News,NaN,POINT(-0.1275 51.507222222222),POINT(2.3513888888889 48.856944444444)


### In case of clusters of other keywords + wkt

In [ ]:
transformed_data = []

for cluster, group_df in df.groupby('Cluster'):
    group_df = group_df.sort_values(by='Date')
    group_df['coord'] = group_df['lat'] + ", " + group_df['long']
    group_df['origin'] = group_df['coord']
    group_df['destination'] = group_df['coord'].shift(-1).fillna(group_df['wkt'].iloc[-1])
    transformed_data.append(group_df)

# Concatenate all transformed groups
transformed_df = pd.concat(transformed_data)

### Without cluster but with coordinates

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by='Date')

In [ ]:
df['origin_lat'] = df['lat']
df['origin_long'] = df['long']

In [ ]:
df['dest_lat'] = df['lat'].shift(-1).fillna(df['lat'].iloc[-1])
df['dest_long'] = df['long'].shift(-1).fillna(df['long'].iloc[-1])

### Common Operations

In [39]:
transformed_df['Date'] = pd.to_datetime(transformed_df['Date']).dt.strftime('%Y-%m-%dT%H:%M:%S')

In [31]:
transformed_df[['origin_longitude', 'origin_latitude']] = transformed_df['origin'].str.extract(r'POINT\(([^ ]+) ([^ ]+)\)')

In [32]:
transformed_df[['destination_longitude', 'destination_latitude']] = transformed_df['destination'].str.extract(r'POINT\(([^ ]+) ([^ ]+)\)')

In [33]:
transformed_df.drop(['destination', 'origin', 'wkt'], axis=1, inplace=True)

In [35]:
# Calculate the duration for each movement
transformed_df['Date'] = pd.to_datetime(transformed_df['Date'])
transformed_df['Duration'] = transformed_df.groupby('Cluster')['Date'].diff().fillna(pd.Timedelta(seconds=0))

### Only for Leopard data

In [54]:
transformed_df['City'] = transformed_df['City'].replace('New York', 'New York City')

### Count cities and transform dates in datetimem

In [55]:
transformed_df['city_number'] = transformed_df.groupby(['City'])['City'].transform('count')

In [56]:
transformed_df['Date'] = pd.to_datetime(transformed_df['Date']).dt.strftime('%Y-%m-%dT%H:%M:%S')

In [58]:
transformed_df.to_csv('foursquare.csv', index=False)